In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from networkx.generators.degree_seq import random_degree_sequence_graph
from networkx.algorithms.graphical import is_graphical
from networkx.utils.random_sequence import powerlaw_sequence
import matplotlib.pyplot as plt
plt.style.use('../figures/norm.mplstyle')
from ldpc.code_util import *
from ldpc.mod2 import *
from bposd.hgp import hgp
from bposd.css import css_code, compute_code_distance
import sys, os
from timeit import default_timer as timer
import cairo
import cmath

def read_pc(filepath):
    """
    Read parity check matrix from file.
    """
    with open(filepath, 'r') as f:
        lines = f.readlines()
    pc = []
    for line in lines:
        row = [int(x) for x in line.split()]
        pc.append(row)
    return np.array(pc, dtype=np.uint8)

def get_classical_code_distance(h):
    if rank(h) == h.shape[1]:
        print('Code is full rank, no codewords')
        return np.inf
    else:
        start = timer()
        print('Code is not full rank, there are codewords')
        print('Computing codeword space basis ...')
        ker = nullspace(h)
        print('debug: ker = ', ker)
        end = timer()
        print(f'Elapsed time for computing codeword space basis: {end-start} seconds', flush=True)
        print('len of ker: ', len(ker))
        print('Start finding minimum Hamming weight while buiding codeword space ...')
        start = end
        # @jit
        def find_min_weight_while_build(matrix):
            span = []
            min_hamming_weight = np.inf
            for ir, row in enumerate(matrix):
                print('debug: ir = ', ir, 'current min_hamming_weight = ', min_hamming_weight, flush=True)  # debug
                row_hamming_weight = np.sum(row)
                if row_hamming_weight < min_hamming_weight:
                    min_hamming_weight = row_hamming_weight
                temp = [row]
                for element in span:
                    newvec = (row + element) % 2
                    temp.append(newvec)
                    newvec_hamming_weight = np.sum(newvec)
                    if newvec_hamming_weight < min_hamming_weight:
                        min_hamming_weight = newvec_hamming_weight
                span = list(np.unique(temp + span, axis=0))
            assert len(span) == 2**len(matrix) - 1
            return min_hamming_weight
        min_hamming_weight = find_min_weight_while_build(ker)
        end = timer()
        print(f'Elapsed time for finding minimum Hamming weight while buiding codeword space : {end-start} seconds', flush=True)
        
        return min_hamming_weight

In [ ]:
def adjust_color_brightness(rgb, factor):
    return tuple(min(255, int(c * factor)) for c in rgb)
# Function to convert HEX to RGB
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

blue_hex = '#6167AF'
blue_rgb = hex_to_rgb(blue_hex)
# Generate gradient colors
blues_grad = [adjust_color_brightness(blue_rgb, 1 + i * 0.8) for i in range(2)]
# Convert RGB back to HEX for plotting
blues_hex = ['#' + ''.join(f'{c:02x}' for c in color) for color in blues_grad]

red_hex = '#F15B5B'
red_rgb = hex_to_rgb(red_hex)
# Generate gradient colors
reds_grad = [adjust_color_brightness(red_rgb, 1 + i * 0.6) for i in range(3)]
# Convert RGB back to HEX for plotting
reds_hex = ['#' + ''.join(f'{c:02x}' for c in color) for color in reds_grad]

In [ ]:
truncate = 0.2
namples = 100
rng = np.random.default_rng(0)
start = 1
end = -1
step = 3
############################################################################################################
# global settings
############################################################################################################

deglo = 5
degup = 7
seeds = range(21)
noprledge = True
noselfloop = True

def sample_logical_ops(logical_basis, nsamples_logical):
    rng = np.random.default_rng(0)
    k = logical_basis.shape[0]
    logical_ops = np.zeros((nsamples_logical, logical_basis.shape[1]), dtype=int)
    # sample size_logicals logical operators, each of which is a linear combination of logical_basis
    for i in range(nsamples_logical):
        while True:
            coeffs = rng.choice([0, 1], size=k).reshape(1, -1)
            if not np.all(coeffs==0):
                break
        logical_ops[i] = np.mod(coeffs@logical_basis, 2)
    return logical_ops

fig, ax = plt.subplots()

n = 300
error_weights_full = []
synd_weights_min_full = []
for seed in seeds:
    readdir = '/Users/yitan/Google Drive/My Drive/from_cannon/qmemory_simulation/data/laplacian_code'
    readname = f'hclassical_configurationmodel_n={n}_deglo={deglo}_degup={degup}_noprledge={noprledge}_noselfloop={noselfloop}_seed={seed}.txt'
    h = read_pc(os.path.join(readdir, readname))
    logical_basis = nullspace(h)
    logical_ops = sample_logical_ops(logical_basis, nsamples_logical=1)
    logical_op = logical_ops[0]
    posones_logical = np.where(logical_op==1)[0]
    error_weights = np.arange(0, int(truncate*n)+1, 1)
    # error_weights = np.arange(0, len(posones_logical)+1, 1)
    synd_weights_list = []
    for error_weight in error_weights:
        error_vecs = np.zeros((nsamples, h.shape[1]), dtype=int)
        for i in range(nsamples):
            posones_error = rng.choice(posones_logical, size=error_weight, replace=False)
            error_vecs[i, posones_error] = 1
        synd_vecs = np.mod(error_vecs@(h.T), 2)
        synd_weights = np.sum(synd_vecs, axis=1)
        synd_weights_list.append(synd_weights)
    synd_weights_list = np.array(synd_weights_list)
    synd_weights_min = np.min(synd_weights_list, axis=1)
    # num_half = int(len(synd_weights_min)/2)
    # error_weights = error_weights[:num_half]
    # synd_weights_min = synd_weights_min[:num_half]
    error_weights_full.append(error_weights)
    synd_weights_min_full.append(synd_weights_min)
max_error_weight = np.max([error_weights[-1] for error_weights in error_weights_full])
synd_weight_min_avg = np.zeros(max_error_weight+1)
for error_weight in range(max_error_weight+1):
    nonzero_synd_weights = []
    for i in range(len(seeds)):
        if error_weight < len(error_weights_full[i]):
            nonzero_synd_weights.append(synd_weights_min_full[i][error_weight])
    synd_weight_min_avg[error_weight] = np.mean(nonzero_synd_weights)
ax.scatter(np.arange(max_error_weight+1)[start:end:step] / n, synd_weight_min_avg[start:end:step] / n, c=reds_hex[-1], edgecolors='k', zorder=10)
# ax1.plot(np.arange(max_error_weight+1) / n, synd_weight_min_avg / n, '--', color='gray')


n = 500
error_weights_full = []
synd_weights_min_full = []
for seed in seeds:
    readdir = '/Users/yitan/Google Drive/My Drive/from_cannon/qmemory_simulation/data/laplacian_code'
    readname = f'hclassical_configurationmodel_n={n}_deglo={deglo}_degup={degup}_noprledge={noprledge}_noselfloop={noselfloop}_seed={seed}.txt'
    h = read_pc(os.path.join(readdir, readname))
    logical_basis = nullspace(h)
    logical_ops = sample_logical_ops(logical_basis, nsamples_logical=1)
    logical_op = logical_ops[0]
    posones_logical = np.where(logical_op==1)[0]
    error_weights = np.arange(0, int(truncate*n)+1, 1)
    # error_weights = np.arange(0, len(posones_logical)+1, 1)
    synd_weights_list = []
    for error_weight in error_weights:
        error_vecs = np.zeros((nsamples, h.shape[1]), dtype=int)
        for i in range(nsamples):
            posones_error = rng.choice(posones_logical, size=error_weight, replace=False)
            error_vecs[i, posones_error] = 1
        synd_vecs = np.mod(error_vecs@(h.T), 2)
        synd_weights = np.sum(synd_vecs, axis=1)
        synd_weights_list.append(synd_weights)
    synd_weights_list = np.array(synd_weights_list)
    synd_weights_min = np.min(synd_weights_list, axis=1)
    # num_half = int(len(synd_weights_min)/2)
    # error_weights = error_weights[:num_half]
    # synd_weights_min = synd_weights_min[:num_half]
    error_weights_full.append(error_weights)
    synd_weights_min_full.append(synd_weights_min)
max_error_weight = np.max([error_weights[-1] for error_weights in error_weights_full])
synd_weight_min_avg = np.zeros(max_error_weight+1)
for error_weight in range(max_error_weight+1):
    nonzero_synd_weights = []
    for i in range(len(seeds)):
        if error_weight < len(error_weights_full[i]):
            nonzero_synd_weights.append(synd_weights_min_full[i][error_weight])
    synd_weight_min_avg[error_weight] = np.mean(nonzero_synd_weights)
ax.scatter(np.arange(max_error_weight+1)[start:end:step] / n, synd_weight_min_avg[start:end:step] / n, c=reds_hex[-2], edgecolors='k', zorder=10)
# ax.plot(np.arange(max_error_weight+1) / n, synd_weight_min_avg / n, '--', color='gray')

############################################################################################################
# seperator between Laplacian and typical LDPC
############################################################################################################

readdir = '/Users/yitan/Google Drive/My Drive/from_cannon/qmemory_simulation/data/ldpc_code'
deg_bit, deg_check = 4, 5
noprledge = True

size = 60
n, m = deg_check*size, deg_bit*size
error_weights_full = []
synd_weights_min_full = []
for seed in seeds:
    readname = f'hclassical_config_model_nonlocal_n={n}_m={m}_deg_bit={deg_bit}_deg_check={deg_check}_noprledge={noprledge}_seed={seed}.txt'
    h = read_pc(os.path.join(readdir, readname))
    logical_basis = nullspace(h)
    logical_ops = sample_logical_ops(logical_basis, nsamples_logical=1)
    logical_op = logical_ops[0]
    posones_logical = np.where(logical_op==1)[0]
    # error_weights = np.arange(0, len(posones_logical)+1, 1)
    error_weights = np.arange(0, int(truncate*n)+1, 1)
    synd_weights_list = []
    for error_weight in error_weights:
        error_vecs = np.zeros((nsamples, h.shape[1]), dtype=int)
        for i in range(nsamples):
            posones_error = rng.choice(posones_logical, size=error_weight, replace=False)
            error_vecs[i, posones_error] = 1
        synd_vecs = np.mod(error_vecs@(h.T), 2)
        synd_weights = np.sum(synd_vecs, axis=1)
        synd_weights_list.append(synd_weights)
    synd_weights_list = np.array(synd_weights_list)
    synd_weights_min = np.min(synd_weights_list, axis=1)
    # num_half = int(len(synd_weights_min)/2)
    # error_weights = error_weights[:num_half]
    # synd_weights_min = synd_weights_min[:num_half]
    error_weights_full.append(error_weights)
    synd_weights_min_full.append(synd_weights_min)
max_error_weight = np.max([error_weights[-1] for error_weights in error_weights_full])
synd_weight_min_avg = np.zeros(max_error_weight+1)
for error_weight in range(max_error_weight+1):
    nonzero_synd_weights = []
    for i in range(len(seeds)):
        if error_weight < len(error_weights_full[i]):
            nonzero_synd_weights.append(synd_weights_min_full[i][error_weight])
    synd_weight_min_avg[error_weight] = np.mean(nonzero_synd_weights)
ax.scatter(np.arange(max_error_weight+1)[start:end:step] / n, synd_weight_min_avg[start:end:step] / m, c=blues_hex[-1], edgecolors='k', zorder=10)
# ax.plot(np.arange(max_error_weight+1) / n, synd_weight_min_avg / m, '--', color='gray')



size = 100
n, m = deg_check*size, deg_bit*size
error_weights_full = []
synd_weights_min_full = []
for seed in seeds:
    readname = f'hclassical_config_model_nonlocal_n={n}_m={m}_deg_bit={deg_bit}_deg_check={deg_check}_noprledge={noprledge}_seed={seed}.txt'
    h = read_pc(os.path.join(readdir, readname))
    logical_basis = nullspace(h)
    logical_ops = sample_logical_ops(logical_basis, nsamples_logical=1)
    logical_op = logical_ops[0]
    posones_logical = np.where(logical_op==1)[0]
    # error_weights = np.arange(0, len(posones_logical)+1, 1)
    error_weights = np.arange(0, int(truncate*n)+1, 1)
    synd_weights_list = []
    for error_weight in error_weights:
        error_vecs = np.zeros((nsamples, h.shape[1]), dtype=int)
        for i in range(nsamples):
            posones_error = rng.choice(posones_logical, size=error_weight, replace=False)
            error_vecs[i, posones_error] = 1
        synd_vecs = np.mod(error_vecs@(h.T), 2)
        synd_weights = np.sum(synd_vecs, axis=1)
        synd_weights_list.append(synd_weights)
    synd_weights_list = np.array(synd_weights_list)
    synd_weights_min = np.min(synd_weights_list, axis=1)
    # num_half = int(len(synd_weights_min)/2)
    # error_weights = error_weights[:num_half]
    # synd_weights_min = synd_weights_min[:num_half]
    error_weights_full.append(error_weights)
    synd_weights_min_full.append(synd_weights_min)
max_error_weight = np.max([error_weights[-1] for error_weights in error_weights_full])
synd_weight_min_avg = np.zeros(max_error_weight+1)
for error_weight in range(max_error_weight+1):
    nonzero_synd_weights = []
    for i in range(len(seeds)):
        if error_weight < len(error_weights_full[i]):
            nonzero_synd_weights.append(synd_weights_min_full[i][error_weight])
    synd_weight_min_avg[error_weight] = np.mean(nonzero_synd_weights)
ax.scatter(np.arange(max_error_weight+1)[start:end:step] / n, synd_weight_min_avg[start:end:step] / m, c=blues_hex[-2], edgecolors='k', zorder=10)
# ax2.plot(np.arange(max_error_weight+1) / n, synd_weight_min_avg / m, '--', color='gray')

ax.set_xlim(0,0.22)
ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4])
ax.set_xticks([0, 0.05, 0.1, 0.15, 0.2])
fig.set_size_inches(5, 5)
fig.savefig(f'../figures/laplacian_typical_ldpc_confinement_truncate.pdf')
plt.show()